In [ ]:
import json
from datetime import datetime

In [2]:
with open("./data/activity.json", "r") as file:
    data = [json.loads(line) for line in file]

FileNotFoundError: [Errno 2] No such file or directory: './data/activity.json'

In [1]:

import matplotlib.pyplot as plt
from collections import Counter


In [ ]:

# Extracting all button click actions
button_clicks = []

for record in data:
    clicks = record.get("activity_log", {}).get("buttonClicks", [])
    for click in clicks:
        button_clicks.append(click.get("action"))

# Counting the frequency of each button click action
button_click_counts = Counter(button_clicks)

# Counting the frequency of each key press
key_counts = Counter(all_post_edit_keys)

# Plotting the key counts
plt.figure(figsize=(10, 5))
plt.bar(key_counts.keys(), key_counts.values(), color='blue')
plt.title('Frequency of Key Strokes After Content Generation')
plt.xlabel('Key')
plt.ylabel('Count')
plt.grid(axis='y')
plt.show()

# Plotting the button click counts
plt.figure(figsize=(10, 5))
plt.bar(button_click_counts.keys(), button_click_counts.values(), color='green')
plt.title('Frequency of Button Clicks')
plt.xlabel('Button Click Action')
plt.ylabel('Count')
plt.grid(axis='y')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()
